In [ ]:
"""
This module defines a structured judicial reasoning agent using LangGraph.

⚖️ DESIGN PRINCIPLES:
- The agent NEVER accesses raw embeddings or documents.
- The agent NEVER hallucinates law or facts.
- All legal content MUST come from civil_law_rag_tool.
- All factual content MUST come from case_documents_rag_tool.
- The agent produces structured analytical assistance.
- The agent NEVER issues a final ruling.

This architecture is production-ready and modular for startup deployment.
"""

from __future__ import annotations

from typing import TypedDict, List, Dict, Any, Optional
from langgraph.graph import StateGraph, END
from langchain_core.runnables import Runnable
from dataclasses import dataclass


# ============================================================
# ======================= STATE ===============================
# ============================================================

class CaseReasoningState(TypedDict, total=False):
    # Inputs
    judge_query: str
    case_summary: Dict[str, Any]

    # Phase 1: Issue Identification
    identified_issues: List[str]

    # Phase 2: Element Decomposition
    decomposed_elements: Dict[str, List[str]]  # issue -> elements

    # Phase 3: Retrieval
    law_retrievals: Dict[str, Any]             # element -> law articles
    fact_retrievals: Dict[str, Any]            # element -> fact excerpts

    # Phase 4: Evaluation
    element_statuses: Dict[str, str]           # element -> Established / etc
    reasoning_blocks: Dict[str, str]           # element -> structured reasoning

    # Phase 5: Validation
    validation_flags: Dict[str, bool]

    # Phase 6: Output
    final_report: str
    confidence_score: str


# ============================================================
# ======================= TOOL INTERFACE =====================
# ============================================================

@dataclass
class CaseReasonerDependencies:
    llm: Runnable
    civil_law_rag_tool: Runnable
    case_documents_rag_tool: Runnable


# ============================================================
# ======================= NODE 1 =============================
# ============================================================

def extract_issues_node(state: CaseReasoningState, deps: CaseReasonerDependencies):
    """
    Extract legal issues from structured summary + judge query.

    Why it exists:
    Judicial reasoning begins by identifying legally relevant disputes.
    """

    prompt = f"""
    Based ONLY on the structured case summary and judge query below,
    identify the core legal issues requiring analysis.

    Do NOT invent facts.
    Do NOT analyze yet.
    Only list issues.

    Judge Query:
    {state["judge_query"]}

    Case Summary:
    {state["case_summary"]}

    Output as a Python list.
    """

    response = deps.llm.invoke(prompt)

    return {"identified_issues": response}


# ============================================================
# ======================= NODE 2 =============================
# ============================================================

def decompose_issue_node(state: CaseReasoningState, deps: CaseReasonerDependencies):
    """
    Decompose each legal issue into required legal elements.

    Why:
    Judicial analysis operates element-by-element.
    """

    decomposed = {}

    for issue in state["identified_issues"]:
        prompt = f"""
        Decompose the following legal issue into its required legal elements.
        Issue: {issue}

        Return a Python list of elements.
        """

        elements = deps.llm.invoke(prompt)
        decomposed[issue] = elements

    return {"decomposed_elements": decomposed}


# ============================================================
# ======================= NODE 3 =============================
# ============================================================

def retrieve_law_node(state: CaseReasoningState, deps: CaseReasonerDependencies):
    """
    Retrieve governing law for each element.

    STRICT RULE:
    Must use civil_law_rag_tool.
    """

    law_results = {}

    for issue, elements in state["decomposed_elements"].items():
        for element in elements:
            law_results[element] = deps.civil_law_rag_tool.invoke(
                f"Legal rule governing: {element}"
            )

    return {"law_retrievals": law_results}


# ============================================================
# ======================= NODE 4 =============================
# ============================================================

def retrieve_facts_node(state: CaseReasoningState, deps: CaseReasonerDependencies):
    """
    Retrieve factual support for each element.

    STRICT RULE:
    Must use case_documents_rag_tool.
    """

    fact_results = {}

    for issue, elements in state["decomposed_elements"].items():
        for element in elements:
            fact_results[element] = deps.case_documents_rag_tool.invoke(
                f"Facts relevant to: {element}"
            )

    return {"fact_retrievals": fact_results}


# ============================================================
# ======================= NODE 5 =============================
# ============================================================

def evaluate_element_node(state: CaseReasoningState, deps: CaseReasonerDependencies):
    """
    Classify element status:
    - Established
    - Not Established
    - Disputed
    - Insufficient Evidence

    Based ONLY on retrieved law and facts.
    """

    statuses = {}
    reasoning_blocks = {}

    for element in state["law_retrievals"].keys():
        prompt = f"""
        Evaluate the element strictly based on:

        LAW:
        {state["law_retrievals"][element]}

        FACTS:
        {state["fact_retrievals"][element]}

        Classify status:
        Established / Not Established / Disputed / Insufficient Evidence

        Provide structured reasoning.
        """

        result = deps.llm.invoke(prompt)

        statuses[element] = result["status"]
        reasoning_blocks[element] = result["reasoning"]

    return {
        "element_statuses": statuses,
        "reasoning_blocks": reasoning_blocks
    }


# ============================================================
# ======================= NODE 6 =============================
# ============================================================

def apply_law_node(state: CaseReasoningState, deps: CaseReasonerDependencies):
    """
    Applies legal rules to classified facts.

    This step ensures structured judicial logic.
    """

    # This node relies on structured reasoning already generated
    return {}


# ============================================================
# ======================= NODE 7 =============================
# ============================================================

def generate_counterarguments_node(state: CaseReasoningState, deps: CaseReasonerDependencies):
    """
    Generate arguments for both sides.

    Why:
    Judges must consider adversarial structure.
    """

    prompt = f"""
    Based ONLY on structured element analysis:

    {state["reasoning_blocks"]}

    Generate:
    - Plaintiff strongest arguments
    - Defendant strongest arguments
    """

    counterarguments = deps.llm.invoke(prompt)

    return {"reasoning_blocks": {**state["reasoning_blocks"], "counterarguments": counterarguments}}


# ============================================================
# ======================= NODE 8 =============================
# ============================================================

def consistency_validator_node(state: CaseReasoningState, deps: CaseReasonerDependencies):
    """
    Validate internal logical consistency.

    If contradiction found, flag for re-evaluation.
    """

    prompt = f"""
    Validate logical consistency of reasoning:

    {state["reasoning_blocks"]}

    Return:
    - consistent: True/False
    - explanation
    """

    validation = deps.llm.invoke(prompt)

    return {"validation_flags": validation}


# ============================================================
# ======================= NODE 9 =============================
# ============================================================

def outcome_framing_node(state: CaseReasoningState, deps: CaseReasonerDependencies):
    """
    Produce final structured analytical report.

    MUST NOT issue ruling.
    """

    report = f"""
    Case Reasoning Report

    I. Identified Issues
    {state["identified_issues"]}

    II. Legal Framework
    {state["law_retrievals"]}

    III. Element-by-Element Analysis
    {state["reasoning_blocks"]}

    IV. Evidence Classification
    {state["element_statuses"]}

    V. Counterarguments
    {state["reasoning_blocks"].get("counterarguments", "")}

    VI. Reasoned Assessment
    Analytical only. No final ruling issued.

    VII. Confidence Level
    """

    confidence = "High" if state["validation_flags"].get("consistent") else "Medium"

    return {
        "final_report": report,
        "confidence_score": confidence
    }


# ============================================================
# ======================= GRAPH BUILDER ======================
# ============================================================

def build_case_reasoner_graph(deps: CaseReasonerDependencies):

    builder = StateGraph(CaseReasoningState)

    builder.add_node("extract_issues", lambda s: extract_issues_node(s, deps))
    builder.add_node("decompose_issue", lambda s: decompose_issue_node(s, deps))
    builder.add_node("retrieve_law", lambda s: retrieve_law_node(s, deps))
    builder.add_node("retrieve_facts", lambda s: retrieve_facts_node(s, deps))
    builder.add_node("evaluate_element", lambda s: evaluate_element_node(s, deps))
    builder.add_node("counterarguments", lambda s: generate_counterarguments_node(s, deps))
    builder.add_node("validate", lambda s: consistency_validator_node(s, deps))
    builder.add_node("outcome", lambda s: outcome_framing_node(s, deps))

    builder.set_entry_point("extract_issues")

    builder.add_edge("extract_issues", "decompose_issue")
    builder.add_edge("decompose_issue", "retrieve_law")
    builder.add_edge("retrieve_law", "retrieve_facts")
    builder.add_edge("retrieve_facts", "evaluate_element")
    builder.add_edge("evaluate_element", "counterarguments")
    builder.add_edge("counterarguments", "validate")
    builder.add_edge("validate", "outcome")
    builder.add_edge("outcome", END)

    return builder.compile()
